<a href="https://colab.research.google.com/github/gopaldewoolkar/AgenticAI/blob/main/chat_with_mysql_using_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langgraph langchain_openai langchain_community langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00


In [12]:
from google.colab import userdata
groq_api_key = userdata.get('groq_api_key')

In [3]:
import requests

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    # Open a local file in binary write mode
    with open("Chinook.db", "wb") as file:
        # Write the content of the response (the file) to the local file
        file.write(response.content)
    print("File downloaded and saved as Chinook.db")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as Chinook.db


In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [7]:
from langchain_core.prompts import ChatPromptTemplate

template ="""
Based on the table schema below, write a sql query that would answer the user's question:
{schema}

Question: {question}
SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [8]:
prompt.format(schema="my schema", question="How many users are there?")

"Human: \nBased on the table schema below, write a sql query that would answer the user's question:\nmy schema\n\nQuestion: How many users are there?\nSQL Query:\n"

In [9]:
def get_schema(_):
  return db.get_table_info()

In [13]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("groq_api_key")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("langsmith_api_key")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "SQL AGENT"

In [10]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 6.8 MB/s eta 0:00:00


In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
import os
from google.colab import userdata

# Set environment variables
os.environ["GROQ_API_KEY"] = userdata.get("groq_api_key")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("langsmith_api_key")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "SQL AGENT"

# Define the prompt template
template = """
Based on the table schema below, write only the SQL query (without any explanation) that would answer the user's question:
{schema}

Question: {question}
SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)

# Function to get schema information
def get_schema(_):
    return db.get_table_info()

# Instantiate the LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

# Define the chain with the updated prompt and output parser
class SQLQueryExtractor(StrOutputParser):
    def parse(self, output_str):
        # Use regex to extract only the SQL code from the response
        import re
        match = re.search(r"(?i)select.*?;", output_str, re.DOTALL)
        return match.group(0).strip() if match else output_str

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop="\nSQL Result:")
    | SQLQueryExtractor()
)

# Invoke the chain
output = sql_chain.invoke({"question": "How many artists are there?"})
print(output)


SELECT COUNT(ArtistId) FROM Artist;


In [25]:
template = """
Based on the schema below, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL_Query: {query}
SQL_Response: {response}
"""
prompt = ChatPromptTemplate.from_template(template)

In [26]:
def run_query(query):
  return db.run(query)

In [27]:
run_query("SELECT COUNT(ArtistId) AS TotalArtists FROM Artist;")

'[(275,)]'

In [30]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response= lambda variables: run_query(variables["query"])
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [31]:
full_chain.invoke({"question": "How many artists are there?"})

'There are 275 artists in the database.'